# Comparation between Elimination Variable in SamIam and Jupyter Bayesian Network

In [46]:
from pgmpy.factors.discrete import TabularCPD
import numpy as np
grade_cpd = TabularCPD(variable='G',
variable_card=3,
values=[[0.3, 0.05, 0.9, 0.5],
[0.4, 0.25, 0.08, 0.3],
[0.3, 0.7, 0.02, 0.2]],
evidence=['I', 'D'],
evidence_card=[2, 2])


In [22]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
student_model = BayesianModel([('D', 'G'),('I', 'G'),('G', 'L'),('I', 'S')])

In [23]:
grade_cpd = TabularCPD(
variable='G',
variable_card=3,
values=[[0.3, 0.05, 0.9, 0.5],
[0.4, 0.25, 0.08, 0.3],
[0.3, 0.7, 0.02, 0.2]],
evidence=['I', 'D'],
evidence_card=[2, 2])

In [24]:
difficulty_cpd = TabularCPD(
variable='D',
variable_card=2,
values=[[0.6, 0.4]])

intel_cpd = TabularCPD(
variable='I',
variable_card=2,
values=[[0.7, 0.3]])

letter_cpd = TabularCPD(
variable='L',
variable_card=2,
values=[[0.1, 0.4, 0.99],
[0.9, 0.6, 0.01]],
evidence=['G'],
evidence_card=[3])



In [25]:
sat_cpd = TabularCPD(
variable='S',
variable_card=2,
values=[[0.95, 0.2],
[0.05, 0.8]],
evidence=['I'],
evidence_card=[2])


In [26]:
student_model.add_cpds(grade_cpd, difficulty_cpd,
intel_cpd, letter_cpd,sat_cpd)

In [27]:
student_model.get_cpds()


[<TabularCPD representing P(G:3 | I:2, D:2) at 0x7ff69cde8a20>,
 <TabularCPD representing P(D:2) at 0x7ff69cde8e80>,
 <TabularCPD representing P(I:2) at 0x7ff69cde8ef0>,
 <TabularCPD representing P(L:2 | G:3) at 0x7ff69cde8cc0>,
 <TabularCPD representing P(S:2 | I:2) at 0x7ff69cde89b0>]

In [28]:
student_model.active_trail_nodes('D')


{'D': {'D', 'G', 'L'}}

In [29]:
student_model.local_independencies('G')

(G _|_ S | D, I)

In [30]:
student_model.get_independencies()

(D _|_ S, I)
(D _|_ I | S)
(D _|_ L | G)
(D _|_ S | I)
(D _|_ L | S, G)
(D _|_ S | L, I)
(D _|_ S, L | G, I)
(D _|_ L | S, G, I)
(D _|_ S | G, L, I)
(G _|_ S | I)
(G _|_ S | D, I)
(G _|_ S | L, I)
(G _|_ S | D, L, I)
(I _|_ D)
(I _|_ D | S)
(I _|_ L | G)
(I _|_ L | S, G)
(I _|_ L | D, G)
(I _|_ L | S, D, G)
(L _|_ S, D, I | G)
(L _|_ S | I)
(L _|_ D, I | S, G)
(L _|_ S, I | D, G)
(L _|_ S | D, I)
(L _|_ S, D | G, I)
(L _|_ I | S, D, G)
(L _|_ D | S, G, I)
(L _|_ S | D, G, I)
(S _|_ D)
(S _|_ L | G)
(S _|_ D, G, L | I)
(S _|_ L | D, G)
(S _|_ G, L | D, I)
(S _|_ D, G | L, I)
(S _|_ D, L | G, I)
(S _|_ G | D, L, I)
(S _|_ L | D, G, I)
(S _|_ D | G, L, I)

In [31]:
student_model.to_markov_model()

In [32]:
def JoinDistribution(model, arr_del, evidence, result):

    dim_result = 1
    _len_result = student_model.get_cpds(result).values.shape
    for n in range(len(_len_result)):
        dim_result *= _len_result[n]
    _result = student_model.get_cpds(result).values.reshape((dim_result,1))

    for k in range(len(arr_del)):
        
        dim_del = 1
        _len_del = student_model.get_cpds(arr_del[k]).values.shape
        for m in range(len(_len_del)):
            dim_del *=_len_del[m]
        _del = student_model.get_cpds(arr_del[k]).values.reshape((dim_del,1))
        
        #del
        for i in range(dim_del):
            if i%2 == 0:
                active = True
            else:
                active = False
                
            count = k
            for j in range(dim_result):
                if active:
                    _result[j] = _result[j]*_del[i]
                
                if count == 0:
                    if active:
                        active = False
                    else:
                        active = True
                    count = k
                else:
                    count = count-1

            
    return Reduction(_result, arr_del, evidence)


In [33]:
def JoinDistribution(model, arr_del, result):

    dim_result = 1
    _len_result = student_model.get_cpds(result).values.shape
    for n in range(len(_len_result)):
        dim_result *= _len_result[n]
    _result = student_model.get_cpds(result).values.reshape((dim_result,1))
        
    for k in range(len(arr_del)):
        
        dim_del = 1
        _len_del = student_model.get_cpds(arr_del[k]).values.shape
        for m in range(len(_len_del)):
            dim_del *=_len_del[m]
        _del = student_model.get_cpds(arr_del[k]).values.reshape((dim_del,1))
        
        for i in range(dim_del):
            if i%2 == 0:
                active = True
            else:
                active = False
                
            count = k
            for j in range(dim_result):
                if active:
                    _result[j] = _result[j]*_del[i]
                
                if count == 0:
                    if active:
                        active = False
                    else:
                        active = True
                    count = k
                else:
                    count = count-1
            
    return _result

In [34]:
def Marginalization(_result, cardinalidad):
    _ret = np.zeros((cardinalidad,1))
    
    longitud = len(_result)
    _ret_pos = 0
    
    for i in range(len(_result)):
        _ret[_ret_pos] += _result[i]
        
        if((i+1)%(longitud/cardinalidad) == 0):
            _ret_pos +=1
            
    return _ret

In [35]:
def Reduction(_result, arr_del, evidence):
    val = evidence.get(arr_del[0])
    longitud = len(_result)
    
    _arr = np.zeros((int(longitud/2),1))
    
    for i in range(longitud):
        if i%2 == val:
            _arr[int(i/2)] = _result[i]
            
    return Renormalization(_arr)


In [36]:
def Renormalization(_arr):
    suma = np.sum(_arr)
    return _arr/suma

In [40]:
x = student_model.get_cpds("S").values.shape

In [41]:
student_model.get_cpds("G").values

array([[[ 0.3 ,  0.05],
        [ 0.9 ,  0.5 ]],

       [[ 0.4 ,  0.25],
        [ 0.08,  0.3 ]],

       [[ 0.3 ,  0.7 ],
        [ 0.02,  0.2 ]]])

Example Query(Grade)

In [49]:
arr = JoinDistribution(student_model, ["D", "I"], "G")
arr

array([[ 0.05292 ],
       [ 0.00392 ],
       [ 0.02916 ],
       [ 0.0072  ],
       [ 0.07056 ],
       [ 0.0196  ],
       [ 0.002592],
       [ 0.00432 ],
       [ 0.05292 ],
       [ 0.05488 ],
       [ 0.000648],
       [ 0.00288 ]])

In [47]:

Marginalization(arr, student_model.get_cardinality("G"))

array([[ 0.362 ],
       [ 0.2884],
       [ 0.3496]])